<a href="https://colab.research.google.com/github/cusiandrea/datascienceportfolio/blob/main/Virtual_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Virtual assistant

The goal is to implement a memory mechanism in order to **turn a LLM into a virtual assistant**. The assistant must receive prompt and return output in Italian.

In [3]:
#improve processing speed
!pip install accelerate

In order to download the LLMs needed, the notebook is logged in Hugging Face and the Transformers library is imported.

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
import transformers
import torch

At first, the model on which the assistant will be based is defined:
https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

The selected LLM works with English language though. Therefore translations steps are needed, both Italian to English (in order to translate the prompts) and English to Italian (translate the generated text).

The models selected for translation are:

https://huggingface.co/Helsinki-NLP/opus-mt-en-it

https://huggingface.co/Helsinki-NLP/opus-mt-it-en

In [7]:
CHAT_MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
EN_IT_MODEL_NAME = "Helsinki-NLP/opus-mt-en-it"
IT_EN_MODEL_NAME = "Helsinki-NLP/opus-mt-it-en"

Each model is then downloaded, along with the definition of its tokenizer.

In [8]:
chat_tokenizer = transformers.AutoTokenizer.from_pretrained(CHAT_MODEL_NAME)
chat_model = transformers.AutoModelForCausalLM.from_pretrained(CHAT_MODEL_NAME)

en_it_tokenizer = transformers.AutoTokenizer.from_pretrained(EN_IT_MODEL_NAME)
en_it_model = transformers.MarianMTModel.from_pretrained(EN_IT_MODEL_NAME)

it_en_tokenizer = transformers.AutoTokenizer.from_pretrained(IT_EN_MODEL_NAME)
it_en_model = transformers.MarianMTModel.from_pretrained(IT_EN_MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/344M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

The memory is implemented by appending every single step of the conversation to a list (*history*). At every iteration of a while loop, the chatting model receives the complete and updated conversation in the requested template with *apply_chat_template*.

The prompt, input by the user, is translated from Italian to English with the IT->EN model before the generation of the new text, which is the assistant's *answer*. However, the answer contains all the conversation history, therefore only the last item is translated back to Italian with the EN->IT model and eventually printed.

The conversation lasts until the user prompts "grazie, arrivederci" and the while loop ends.

In [9]:
text = ""
history = []

while text.lower()!="grazie, arrivederci":

  text = input("Tu: ")

  tokens = it_en_tokenizer.encode(text, return_tensors="pt")
  output = it_en_model.generate(tokens)
  prompt = it_en_tokenizer.decode(output[0], skip_special_tokens=True)

  history.append({"role":"user","content":prompt})

  tokens = chat_tokenizer.apply_chat_template(history,return_tensors="pt")
  output = chat_model.generate(tokens, max_new_tokens=100, pad_token_id=100)
  answer = chat_tokenizer.decode(output[0], skip_special_tokens=True)
  answer = answer.split("[/INST]")[-1].strip()

  history.append({"role":"assistant","content":answer})

  tokens = en_it_tokenizer.encode(answer, return_tensors="pt")
  output = en_it_model.generate(tokens)
  translation = en_it_tokenizer.decode(output[0], skip_special_tokens=True)

  print("Assistant: "+translation)

Tu: Ciao, potresti per favore dirmi qual è la capitale della Francia?
Assistant: La capitale della Francia è Parigi.
Tu: Grazie. Qual è invece la capitale della Germania?
Assistant: La capitale della Germania è Berlino.
Tu: Perfetto. Quale delle due città è ha più abitanti?
Assistant: A partire dal 2021, Berlino conta circa 3,7 milioni di abitanti, mentre Parigi conta circa 2,1 milioni di abitanti. Pertanto, Berlino ha la maggior parte degli abitanti delle due città.
Tu: Bene. Ora potresti per favore raccontare una barzelletta?
Assistant: Certo, ecco una battuta per te: perché gli scienziati non si fidano degli atomi? Perché inventano tutto!
Tu: grazie, arrivederci
Assistant: Non c'e' di che, buona giornata!


Using 3 LLMs and the help of a while loop, a **simple but functioning** virtual assistand is defined.